In [ ]:
from network import *

In [ ]:
def predict_device_with_unknown_detection(model, iq_file_path, scalers, device, confidence_threshold=0.7):
    """预测设备类型，包含未知设备检测"""
    # 读取数据
    data = load_iq_file(iq_file_path)
    
    # 分段数据
    segment_length = 1024  # 与训练时相同
    num_segments = len(data) // segment_length
    segments = data[:num_segments * segment_length].reshape(num_segments, segment_length)
    
    # 创建数据集和加载器
    dataset = ComplexSignalDataset(segments, np.zeros(len(segments)), scalers[0], scalers[1])
    dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
    
    # 预测
    model.eval()
    all_probs = []
    
    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probs = F.softmax(outputs, dim=1)
            all_probs.append(probs.cpu().numpy())
    
    all_probs = np.vstack(all_probs)
    
    # 获取最高概率及其对应的类别
    max_probs = np.max(all_probs, axis=1)
    predictions = np.argmax(all_probs, axis=1)
    
    # 计算每个段的平均置信度
    avg_confidence = np.mean(max_probs)
    
    # 统计各类别的数量
    unique, counts = np.unique(predictions, return_counts=True)
    result = dict(zip(unique, counts))
    
    # 映射到设备名称
    device_names = ['bladerf', 'hackrf0', 'hackrf1', 'limesdr']
    final_result = {device_names[int(k)]: v for k, v in result.items()}
    
    # 判断是否为未知设备
    if avg_confidence < confidence_threshold:
        return "未知设备", final_result, avg_confidence
    else:
        # 返回出现最多的设备类型
        majority_device = max(final_result, key=final_result.get)
        return majority_device, final_result, avg_confidence

In [ ]:
# 加载模型和标准化器
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_features = 1024  # 与训练时相同
num_classes = 4  # 四种已知设备
model = create_model(num_features, num_classes).to(device)
model.load_state_dict(torch.load('./models/complex_cnn_model.pth'))
scalers = torch.load('./models/scalers.pth')

In [ ]:
# 使用模型预测未知设备
unknown_file = "../dataset/raw data/bladerf/bladerf.iq"
device_type, class_counts, confidence = predict_device_with_unknown_detection(
    model, unknown_file, scalers, device, confidence_threshold=0.7)

print(f"预测的设备类型: {device_type}")
print(f"各类别的预测数量: {class_counts}")
print(f"平均置信度: {confidence:.4f}")

# 如果是已知设备，可以进一步分析
if device_type != "未知设备":
    print(f"这是一个已知设备: {device_type}")
else:
    print("这可能是一个训练数据中未包含的设备！")